# Getting data ready to be used with machine learning

Three main things to do:
- Split the data into features and labels (ususally `X` & `y`)
- Fill (also called imputing) or disregard missing values
- Convert non-numerical values to numerical values (also called feautre encoding)

In [61]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [110]:
car_sales_missing = pd.read_csv('../data/car-sales-extended-missing-data.csv')
car_sales_missing.head(), len(car_sales_missing), car_sales_missing.dtypes

(     Make Colour  Odometer (KM)  Doors    Price
 0   Honda  White        35431.0    4.0  15323.0
 1     BMW   Blue       192714.0    5.0  19943.0
 2   Honda  White        84714.0    4.0  28343.0
 3  Toyota  White       154365.0    4.0  13434.0
 4  Nissan   Blue       181577.0    3.0  14043.0,
 1000,
 Make              object
 Colour            object
 Odometer (KM)    float64
 Doors            float64
 Price            float64
 dtype: object)

In [111]:
car_sales_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

## Split data

### Option 1: Fill missing data with Pandas

In [98]:
# # Fill the "Make" column
# car_sales_missing['Make'] = car_sales_missing['Make'].fillna("missing")
# # Fill the "Colour" column
# car_sales_missing['Colour'] = car_sales_missing['Colour'].fillna("missing")
# # Fill the "Odometer (KM)" column
# car_sales_missing['Odometer (KM)'] = car_sales_missing['Odometer (KM)'].fillna(car_sales_missing['Odometer (KM)'].mean())
# # Fill the "Doors" column
# car_sales_missing['Doors'] = car_sales_missing['Doors'].fillna(car_sales_missing['Doors'].mode()[0])
# # Remove rows with missing "Price" values
# car_sales_missing.dropna(subset=['Price'], inplace=True)

# car_sales_missing.isna().sum()


### Option 2: Filling missing data with Sklearn (Recommended)

In [112]:
# Drop the rows with missing price values
car_sales_missing.dropna(subset=['Price'], inplace=True)

X = car_sales_missing.drop('Price', axis=1)
y = car_sales_missing['Price']

# Fill missing values with sklearn's SimpleImputer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
# Define the columns to be imputed
categorical_cols = ['Make', 'Colour']
# Create a column transformer to handle different types of data
column_transformer = ColumnTransformer(
    transformers=[
        ('cat_imputer', SimpleImputer(strategy='constant', fill_value="missing"), categorical_cols),
        ('num_imputer', SimpleImputer(strategy='mean'), ['Odometer (KM)']),
        ('door_imputer', SimpleImputer(strategy='constant', fill_value=4), ['Doors']),
    ],
    remainder='passthrough'  # Keep other columns as they are
)
# Transform the data with missing values
filled_X = column_transformer.fit_transform(X)
filled_X.shape, filled_X[:5]

((950, 4),
 array([['Honda', 'White', 35431.0, 4.0],
        ['BMW', 'Blue', 192714.0, 5.0],
        ['Honda', 'White', 84714.0, 4.0],
        ['Toyota', 'White', 154365.0, 4.0],
        ['Nissan', 'Blue', 181577.0, 3.0]], dtype=object))

In [113]:
car_sales_filled = pd.DataFrame(filled_X, columns=X.columns)
car_sales_filled, car_sales_filled.isna().sum()

(        Make Colour Odometer (KM) Doors
 0      Honda  White       35431.0   4.0
 1        BMW   Blue      192714.0   5.0
 2      Honda  White       84714.0   4.0
 3     Toyota  White      154365.0   4.0
 4     Nissan   Blue      181577.0   3.0
 ..       ...    ...           ...   ...
 945   Toyota  Black       35820.0   4.0
 946  missing  White      155144.0   3.0
 947   Nissan   Blue       66604.0   4.0
 948    Honda  White      215883.0   4.0
 949   Toyota   Blue      248360.0   4.0
 
 [950 rows x 4 columns],
 Make             0
 Colour           0
 Odometer (KM)    0
 Doors            0
 dtype: int64)

## Convert non-numerical data to numerical data

In [115]:
# Turn categorical variables into numbers
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# Define the column transformer
categorical_features = ['Make', 'Colour', 'Doors']
column_transformer = ColumnTransformer(
    transformers=[
        ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough')
# Fit the column transformer to the training data
transformed_X = column_transformer.fit_transform(car_sales_filled)
transformed_X


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3800 stored elements and shape (950, 15)>

In [116]:
pd.DataFrame(transformed_X)

,0
0,<Compressed Sparse Row sparse matrix of dtype ...
1,<Compressed Sparse Row sparse matrix of dtype ...
2,<Compressed Sparse Row sparse matrix of dtype ...
3,<Compressed Sparse Row sparse matrix of dtype ...
4,<Compressed Sparse Row sparse matrix of dtype ...
...,...
945,<Compressed Sparse Row sparse matrix of dtype ...
946,<Compressed Sparse Row sparse matrix of dtype ...
947,<Compressed Sparse Row sparse matrix of dtype ...
948,<Compressed Sparse Row sparse matrix of dtype ...


In [117]:
# Split data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(transformed_X, y, test_size=0.2, random_state=42)

In [118]:
from sklearn.ensemble import RandomForestRegressor
# Create a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
# Fit the model to the training data
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [119]:
model.score(X_train, y_train), model.score(X_test, y_test)

(0.8805115715925956, 0.2084356464002628)